In [1]:
using DataFrames
using JLD2
using CairoMakie
using Printf

In [3]:
df1 = load(@__DIR__()*"/output/ks_dists_range_1_48.jld2", "ks_table");
df2 = load(@__DIR__()*"/output/ks_dists_range_49_58.jld2", "ks_table");
df3 = load(@__DIR__()*"/output/ks_dists_range_59_68.jld2", "ks_table");
df4 = load(@__DIR__()*"/output/ks_dists_range_69_76.jld2", "ks_table");
df_big = vcat(df1, df2, df3, df4);

In [16]:
df_t = load(@__DIR__()*"/output/ks_dists_DDN_smag_lf.jld2", "ks_table");
for c in names(df_big)
    if !(c in names(df_t))
        df_t[!,c] = repeat(["NA"],size(df_t, 1))
    end
end

In [17]:
df_big = vcat(df_big, df_t);

In [19]:
function print_df2latex(df, cols)
    
    #print latex table
    for col in cols
        if contains(string(col), '_')
            print(replace(string(col), '_' => ' '))
        else
            print(string(col))
        end
        if col != cols[end]
            print(" & ")
        else
            println("\\\\")
        end
    end
    for i in 1:nrow(df)
        for col in cols
            if col == "name"
                print("LRS")
            else
                if col in ["summed_ks", "ks_best_replica", "ks_worst_replica"]
                    @printf("%.2f", df[i, col])
                else
                    if contains(string(df[i, col]), '_')
                        print(replace(string(df[i, col]), '_' => ' '))
                    else
                        print(string(df[i, col]))
                    end
                end
            end
            if col != "ks_worst_replica"
                print(" & ")
            else
                println("\\\\")
            end
        end
        
    end
end

print_df2latex (generic function with 1 method)

In [20]:
function f(x)
    if isnothing(x)
        return NaN
    else
        return x[1]
    end
end

df_big[!,"summed_ks"] = map(f, df_big[!,"ks_dist_ensemble"]);


In [23]:
function mi(x)
    if isnothing(x)
        return NaN
    else
        return minimum(stack(x)[1,:])
    end
end

function ma(x)
    if isnothing(x)
        return NaN
    else
        return maximum(stack(x)[1,:])
    end
end

df_big[!,"ks_best_replica"] = map(mi, df_big[!,"ks_dist_replicas"]);
df_big[!,"ks_worst_replica"] = map(ma, df_big[!,"ks_dist_replicas"]);

In [ ]:
x = df_big[df_big.tracking_noise .== 0.0, :]
x = x[x.model_noise .== :MVG, :]
#print(x[!,["name", "tracking_noise",	"hist_len",	"lambda", "model_noise","n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"]])
x_0 = x[(x.lambda .== "NA") .| (x.lambda .== 0.0), :];
print(x_0[!,["name", "tracking_noise",	"hist_len",	"lambda", "model_noise","n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"]])


19×9 DataFrame
 Row │ name      tracking_noise  hist_len  lambda  model_noise  n_unstable  summed_ks   ks_best_replica  ks_worst_replica 
     │ String    Float64         Int64     Any     Symbol       Any         Float64     Float64          Float64          
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ LinReg1              0.0         5  0.0     MVG          0             0.19812          0.141596          0.370516
   2 │ LinReg25             0.0        20  0.0     MVG          0             0.21629          0.169171          0.406965
   3 │ LinReg50             0.0         0  0.0     MVG          5             0.438475         0.39805           1.29547
   4 │ LinReg52             0.0         1  0.0     MVG          0             0.347861         0.252819          0.509012
   5 │ LinReg54             0.0         3  0.0     MVG          0             0.260293         0.20222           0.450714
   6 │ 

false

In [ ]:
## plot Predictive quality of SGS models  BIG data
x = df_big[df_big.tracking_noise .== 0.0, :]
x = x[x.model_noise .== :MVG, :]
#print(x[!,["name", "tracking_noise",	"hist_len",	"lambda", "model_noise","n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"]])
x_0 = x[(x.lambda .== "NA") .| (x.lambda .== 0.0), :];

fig = Figure()
a = Axis(fig[1, 1], xticks = ([-15, -10, -5, 0, 10, 20, 30, 40, 50 ,60,70,80], ["NoModel", "smag", "DDN", "0", "10", "20", "30", "40", "50" ,"60","70","80"]), xticklabelrotation = pi/4)
x0_unstable = x_0[x_0.n_unstable .>= 1, :]
x0_stable = x_0[x_0.n_unstable .== 0, :]

scatter!(x0_stable.hist_len, x0_stable.summed_ks; color = :blue, label= "ensemble")
rangebars!(x0_stable.hist_len, x0_stable.ks_worst_replica, x0_stable.ks_best_replica; color = :blue, label= "range") 
scatter!(x0_unstable.hist_len, x0_unstable.summed_ks; color = :blue, label= "unstable", markersize = 15, marker = 'X')

axislegend(a, postion=:ct)

a.ylabel="Summed KS-distance"
a.xlabel="SGS model / History length LRS"

fig
save(@__DIR__()*"/../output/figures_paper/linregs/history_length_notracknoise.pdf", fig)
#save(@__DIR__()*"/../output/figures_paper/linregs/history_length_notracknoise.png", fig)

CairoMakie.Screen{PDF}


In [ ]:
## plot Predictive quality of SGS models influence regularization
x = df_big[df_big.tracking_noise .== 0.0, :]
x = x[x.model_noise .== :MVG .&& x.hist_len .!= 50, :]

x_0 = x[x.lambda .== 0.0, :]
x_01 = x[x.lambda .== 0.01, :]
#print(x_01[!,["name", "tracking_noise",	"hist_len",	"lambda", "model_noise","n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"]])
x0_unstable = x_0[x_0.n_unstable .>= 1, :]
x0_stable = x_0[x_0.n_unstable .== 0, :]
x01_unstable = x_01[x_01.n_unstable .>= 1, :]
x01_stable = x_01[x_01.n_unstable .== 0, :]

fig = Figure()
a = Axis(fig[1, 1])

e=scatter!(x0_stable.hist_len, x0_stable.summed_ks; color = :blue, label= "ensemble")
r=rangebars!(x0_stable.hist_len, x0_stable.ks_worst_replica, x0_stable.ks_best_replica; color = :blue, label= "range") 
x=scatter!(x0_unstable.hist_len, x0_unstable.summed_ks; color = :blue, label= "unstable", markersize = 15, marker = 'X')

g=scatter!(x01_stable.hist_len.+0.3, x01_stable.summed_ks; color = :green, label= "ensemble")
rangebars!(x01_stable.hist_len.+0.3, x01_stable.ks_worst_replica, x01_stable.ks_best_replica; color = :green, label= "range") 
scatter!(x01_unstable.hist_len.+0.3, x01_unstable.summed_ks; color = :green, label= "unstable", markersize = 15, marker = 'X')

p = [MarkerElement(color = :black, marker = :circle)]
l = [LineElement(color = :black)]
x = [MarkerElement(color = :black, marker = 'X', markersize = 15)]
axislegend(a, [e, g, p,l,x], [L"$\lambda=0$", L"$\lambda=0.01$", "ensemble", "range", "unstable"])
a.ylabel="Summed KS-distance"
a.xlabel="History length"

fig
save(@__DIR__()*"/../output/figures_paper/linregs/history_length_regularization.pdf", fig)

CairoMakie.Screen{PDF}


In [16]:
### small training data
df1 = load(@__DIR__()*"/output/ks_dists_range_1_48.jld2", "ks_table");
df2 = load(@__DIR__()*"/output/ks_dists_range_49_58.jld2", "ks_table");
df3 = load(@__DIR__()*"/output/ks_dists_range_59_68.jld2", "ks_table");
df4 = load(@__DIR__()*"/output/ks_dists_range_69_76.jld2", "ks_table");
df5 = load(@__DIR__()*"/output/ks_dists_range_77_148.jld2", "ks_table");
df6 = load(@__DIR__()*"/output/ks_dists_range_149_155.jld2", "ks_table");
df_big = vcat(df1, df2, df3, df4, df5, df6);

function f(x)
    if isnothing(x)
        return NaN
    else
        return x[1]
    end
end
df_big[!,"summed_ks"] = map(f, df_big[!,"ks_dist_ensemble"]);
function mi(x)
    if isnothing(x)
        return NaN
    else
        return minimum(stack(x)[1,:])
    end
end

function ma(x)
    if isnothing(x)
        return NaN
    else
        return maximum(stack(x)[1,:])
    end
end

df_big[!,"ks_best_replica"] = map(mi, df_big[!,"ks_dist_replicas"]);
df_big[!,"ks_worst_replica"] = map(ma, df_big[!,"ks_dist_replicas"]);


In [36]:
for lab in [0.0, 0.01, 0.1]
    x = df_big[df_big.tracking_noise .== 0.0, :]
    x = x[x.model_noise .== :MVG, :]
    x = x[x.lambda .== lab, :]
    x = x[x.hist_len .== 5 .|| x.hist_len .== 10, :]

    x.data_size = map(x -> x[2]/400, x.train_range)

    print(x[!,["name", "tracking_noise",	"hist_len",	"lambda", "train_range", "data_size","n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"]])

    x5_unstable = x[x.hist_len .== 5 .&& x.n_unstable .>= 1, :]
    x5_stable = x[x.hist_len .== 5 .&& x.n_unstable .== 0, :]
    x10_unstable = x[x.hist_len .== 10 .&& x.n_unstable .>= 1, :]
    x10_stable = x[x.hist_len .== 10 .&& x.n_unstable .== 0, :]

    fig = Figure(size=(450,350), fontsize=18)
    a = Axis(fig[1, 1])

    e=scatter!(x5_stable.data_size.-0.05, x5_stable.summed_ks; color = :blue, label= "ensemble")
    r=rangebars!(x5_stable.data_size.-0.05, x5_stable.ks_worst_replica, x5_stable.ks_best_replica; color = :blue, label= "range") 
    x=scatter!(x5_unstable.data_size.-0.05, x5_unstable.summed_ks; color = :blue, label= "unstable", markersize = 15, marker = 'X')

    g=scatter!(x10_stable.data_size.+0.05, x10_stable.summed_ks; color = :green, label= "ensemble")
    rangebars!(x10_stable.data_size.+0.05, x10_stable.ks_worst_replica, x10_stable.ks_best_replica; color = :green, label= "range")
    scatter!(x10_unstable.data_size.+0.05, x10_unstable.summed_ks; color = :green, label= "unstable", markersize = 15, marker = 'X')


    p = [MarkerElement(color = :black, marker = :circle)]
    l = [LineElement(color = :black)]
    x = [MarkerElement(color = :black, marker = 'X', markersize = 15)]
    axislegend(a, [e,g,p,l,x], ["hist 5", "hist 10", "ensemble", "range", "unstable"])
    a.ylabel="Summed KS-distance"
    a.xlabel="Training data size"
    a.xticks = ([2.5, 5, 7.5, 10], ["2.5", "5", "7.5", "10"])
    ylims!(a,(0, 1.65))
    fig
    save(@__DIR__()*"/../output/figures_paper/linregs/train_size_lambda_$(lab).pdf", fig)

end



8×10 DataFrame
 Row │ name       tracking_noise  hist_len  lambda   train_range  data_size  n_unstable  summed_ks  ks_best_replica  ks_worst_replica 
     │ String     Float64         Int64     Float64  Tuple…       Float64    Any         Float64    Float64          Float64          
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ LinReg1               0.0         5      0.0  (400, 4000)       10.0  0            0.19812          0.141596          0.370516
   2 │ LinReg64              0.0        10      0.0  (400, 4000)       10.0  0            0.197235         0.213045          0.39229
   3 │ LinReg77              0.0         5      0.0  (400, 2000)        5.0  0            0.78831          0.677683          1.01657
   4 │ LinReg78              0.0         5      0.0  (400, 1000)        2.5  3            0.974445         0.697018          1.11107
   5 │ LinReg95              0.0        10     

In [ ]:
x = df_big[df_big.tracking_noise .== 0.0, :]
x = x[x.model_noise .== :MVG, :]
#print(x[!,["name", "tracking_noise",	"hist_len",	"lambda", "model_noise","n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"]])
#x_0 = x[(x.lambda .== "NA") .| (x.lambda .== 0.0), :];
sort!(df_big, [:summed_ks])
print(df_big[!,["name", "tracking_noise",	"hist_len",	"lambda", "train_range","n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"]])



72×9 DataFrame
 Row │ name       tracking_noise  hist_len  lambda   train_range  n_unstable  summed_ks   ks_best_replica  ks_worst_replica 
     │ String     Float64         Int64     Float64  Tuple…       Any         Float64     Float64          Float64          
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ LinReg91             0.01         5     0.1   (400, 2000)  0             0.219655         0.259369          0.330192
   2 │ LinReg98             0.0         10     0.1   (400, 1000)  1             0.286257         0.268268          0.576984
   3 │ LinReg100            0.0         10     0.01  (400, 1000)  0             0.318127         0.298543          0.38689
   4 │ LinReg89             0.01         5     0.0   (400, 2000)  1             0.324747         0.307192          0.585291
   5 │ LinReg125            0.01        20     0.0   (400, 2000)  0             0.371931         0.347016          

In [16]:
using CSV
CSV.write("./output/data_table_smalldata", df_big[!,["name", "tracking_noise",	"hist_len",	"lambda", "train_range","n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"]])

"./output/data_table_smalldata"

In [ ]:
#### old stuff

In [23]:
sort!(df_big, [:hist_len])
print(df_big[(df_big.tracking_noise .== 0.0) .& (df_big.model_noise .== :MVG).& (df_big.lambda .== 0.0), 
    ["name", "tracking_noise", "train_range","hist_len",	"lambda", "model_noise","n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"]])

16×10 DataFrame
 Row │ name      tracking_noise  train_range  hist_len  lambda   model_noise  n_unstable  summed_ks   ks_best_replica  ks_worst_replica 
     │ String    Float64         Tuple…       Int64     Float64  Symbol       Any         Float64     Float64          Float64          
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ LinReg50             0.0  (400, 4000)         0      0.0  MVG          5             0.438475         0.39805           1.29547
   2 │ LinReg52             0.0  (400, 4000)         1      0.0  MVG          0             0.347861         0.252819          0.509012
   3 │ LinReg54             0.0  (400, 4000)         3      0.0  MVG          0             0.260293         0.20222           0.450714
   4 │ LinReg60             0.0  (400, 4000)         4      0.0  MVG          0             0.235894         0.203795          0.394065
   5 │ LinReg1              0.

In [34]:
df_print=df_big[(df_big.tracking_noise .== 0.0) .& (df_big.model_noise .== :MVG).& (df_big.lambda .== 0.0), ["name", "hist_len",	"lambda" ,"n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"]];
print_df2latex(df_print, ["name", "hist_len","n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"])

name & hist len & n unstable & summed ks & ks best replica & ks worst replica\\
LRS & 0 & 5 & 0.44 & 0.40 & 1.30\\
LRS & 1 & 0 & 0.35 & 0.25 & 0.51\\
LRS & 3 & 0 & 0.26 & 0.20 & 0.45\\
LRS & 4 & 0 & 0.24 & 0.20 & 0.39\\
LRS & 5 & 0 & 0.20 & 0.14 & 0.37\\
LRS & 6 & 0 & 0.25 & 0.21 & 0.35\\
LRS & 7 & 0 & 0.20 & 0.17 & 0.33\\
LRS & 8 & 0 & 0.23 & 0.20 & 0.36\\
LRS & 10 & 0 & 0.20 & 0.21 & 0.39\\
LRS & 20 & 0 & 0.22 & 0.17 & 0.41\\
LRS & 25 & 0 & 0.26 & 0.23 & 0.35\\
LRS & 30 & 0 & 0.19 & 0.17 & 0.38\\
LRS & 40 & 3 & 0.25 & 0.28 & 0.44\\
LRS & 50 & 5 & NaN & NaN & NaN\\
LRS & 60 & 5 & 0.39 & 0.40 & 0.45\\
LRS & 80 & 5 & 1.00 & 0.82 & 1.67\\


In [24]:
sort!(df_big, [:hist_len])
print(df_big[(df_big.tracking_noise .== 0.0) .& (df_big.model_noise .== :MVG).& (df_big.lambda .== 0.01), 
    ["name", "tracking_noise", "train_range","hist_len",	"lambda", "model_noise","n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"]])

16×10 DataFrame
 Row │ name      tracking_noise  train_range  hist_len  lambda   model_noise  n_unstable  summed_ks  ks_best_replica  ks_worst_replica 
     │ String    Float64         Tuple…       Int64     Float64  Symbol       Any         Float64    Float64          Float64          
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ LinReg49             0.0  (400, 4000)         0     0.01  MVG          5            0.756582         0.584644          1.00999
   2 │ LinReg51             0.0  (400, 4000)         1     0.01  MVG          5            1.81989          1.51471           2.65703
   3 │ LinReg53             0.0  (400, 4000)         3     0.01  MVG          2            0.474144         0.295868          0.742556
   4 │ LinReg59             0.0  (400, 4000)         4     0.01  MVG          2            0.493812         0.463838          1.09485
   5 │ LinReg5              0.0  (400, 

In [34]:
sort!(df_big, [:summed_ks])
print(df_big[map(x -> x[2], df_big.train_range) .== 1000,["name", "tracking_noise", "train_range","hist_len",	"lambda", "model_noise","n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"]])

36×10 DataFrame
 Row │ name       tracking_noise  train_range  hist_len  lambda   model_noise  n_unstable  summed_ks  ks_best_replica  ks_worst_replica 
     │ String     Float64         Tuple…       Int64     Float64  Symbol       Any         Float64    Float64          Float64          
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ LinReg98             0.0   (400, 1000)        10     0.1   MVG          1            0.286257         0.268268          0.576984
   2 │ LinReg100            0.0   (400, 1000)        10     0.01  MVG          0            0.318127         0.298543          0.38689
   3 │ LinReg118            0.0   (400, 1000)        20     0.01  MVG          0            0.444614         0.359116          0.574611
   4 │ LinReg80             0.0   (400, 1000)         5     0.1   MVG          4            0.452253         0.338539          0.894778
   5 │ LinReg116            0.

In [26]:
sort!(df_big, [:summed_ks])
print(df_big[map(x -> x[2], df_big.train_range) .== 2000,["name", "tracking_noise", "train_range","hist_len",	"lambda", "model_noise","n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"]])

36×10 DataFrame
 Row │ name       tracking_noise  train_range  hist_len  lambda   model_noise  n_unstable  summed_ks   ks_best_replica  ks_worst_replica 
     │ String     Float64         Tuple…       Int64     Float64  Symbol       Any         Float64     Float64          Float64          
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ LinReg91             0.01  (400, 2000)         5     0.1   MVG          0             0.219655         0.259369          0.330192
   2 │ LinReg89             0.01  (400, 2000)         5     0.0   MVG          1             0.324747         0.307192          0.585291
   3 │ LinReg125            0.01  (400, 2000)        20     0.0   MVG          0             0.371931         0.347016          0.462538
   4 │ LinReg143            0.01  (400, 2000)        30     0.0   MVG          0             0.372526         0.364666          0.448814
   5 │ LinReg131      

In [119]:
x = df_big[df_big.tracking_noise .== 0.0, :]
x = x[x.model_noise .== :MVG, :]
#print(x[!,["name", "tracking_noise",	"hist_len",	"lambda", "model_noise","n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"]])
x_0 = x[x.lambda .== 0.0, :]
x_01 = x[x.lambda .== 0.01, :]

fig = Figure()
a = Axis(fig[1, 1])
x0_unstable = x_0[x_0.n_unstable .>= 1, :]
scatter!(x0_unstable.hist_len, x0_unstable.summed_ks; color = :blue, label= "contains unstable", markersize = 20, marker = 'X')
x01_unstable = x_01[x_01.n_unstable .>= 1, :]
scatter!(x01_unstable.hist_len.+0.3, x01_unstable.summed_ks; color = :green, label= "contains unstable", markersize = 20, marker = 'X')
rangebars!(x_0.hist_len, x_0.ks_worst_replica, x_0.ks_best_replica; color = :blue, label= "λ = 0, range") 
scatter!(x_0.hist_len, x_0.summed_ks; color = :blue, label= "λ = 0, ensemble")
rangebars!(x_01.hist_len.+0.3, x_01.ks_worst_replica, x_01.ks_best_replica; color = :green, label= "λ = 0.01, range") 
scatter!(x_01.hist_len.+0.3, x_01.summed_ks; color = :green, label= "λ = 0.01, ensemble")



axislegend(a)
a.ylabel="Summed KS-distance"
a.xlabel="History length"

fig
save(@__DIR__()*"/../output/figures_paper/linregs/history_length_notracknoise.pdf", fig)
save(@__DIR__()*"/../output/figures_paper/linregs/history_length_notracknoise.png", fig)

CairoMakie.Screen{IMAGE}


In [107]:
print(x_0[!,["name", "tracking_noise",	"hist_len",	"lambda", "model_noise","n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"]])

16×9 DataFrame
 Row │ name      tracking_noise  hist_len  lambda   model_noise  n_unstable  summed_ks   ks_best_replica  ks_worst_replica 
     │ String    Float64         Int64     Float64  Symbol       Any         Float64     Float64          Float64          
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ LinReg68             0.0        30      0.0  MVG          0             0.1938           0.165446          0.38449
   2 │ LinReg64             0.0        10      0.0  MVG          0             0.197235         0.213045          0.39229
   3 │ LinReg1              0.0         5      0.0  MVG          0             0.19812          0.141596          0.370516
   4 │ LinReg72             0.0         7      0.0  MVG          0             0.20265          0.174546          0.331492
   5 │ LinReg25             0.0        20      0.0  MVG          0             0.21629          0.169171          0.406965


In [108]:
print(x_01[!,["name", "tracking_noise",	"hist_len",	"lambda", "model_noise","n_unstable","summed_ks", "ks_best_replica", "ks_worst_replica"]])

16×9 DataFrame
 Row │ name      tracking_noise  hist_len  lambda   model_noise  n_unstable  summed_ks  ks_best_replica  ks_worst_replica 
     │ String    Float64         Int64     Float64  Symbol       Any         Float64    Float64          Float64          
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ LinReg69             0.0         6     0.01  MVG          0            0.188755         0.224169          0.335192
   2 │ LinReg5              0.0         5     0.01  MVG          0            0.232814         0.175171          0.442689
   3 │ LinReg29             0.0        20     0.01  MVG          0            0.355061         0.295993          0.482463
   4 │ LinReg61             0.0         8     0.01  MVG          0            0.38857          0.313292          0.497938
   5 │ LinReg71             0.0         7     0.01  MVG          0            0.414955         0.328142          0.526312
   6 │

In [ ]:
using Printf
#print latex table
cols = ["name", "tracking_noise",	"hist_len",	"lambda", "model_noise","summed_ks", "ks_best_replica", "ks_worst_replica"]
for col in cols
    if contains(string(col), '_')
        print(replace(string(col), '_' => ' '))
    else
        print(string(col))
    end
    if col != cols[end]
        print(" & ")
    else
        println("\\\\")
    end
end
for i in 1:nrow(df)
    for col in cols
        if col in ["summed_ks", "ks_best_replica", "ks_worst_replica"]
            @printf("%.2f", df[i, col])
        else
            if contains(string(df[i, col]), '_')
                print(replace(string(df[i, col]), '_' => ' '))
            else
                print(string(df[i, col]))
            end
        end
        if col != "ks_worst_replica"
            print(" & ")
        else
            println("\\\\")
        end
    end
    
end

name & tracking noise & hist len & lambda & model noise & summed ks & ks best replica & ks worst replica\\
LinReg3 & 0.0 & 5 & 0.1 & MVG & 0.19 & 0.22 & 0.35\\
LinReg1 & 0.0 & 5 & 0.0 & MVG & 0.20 & 0.14 & 0.37\\
LinReg25 & 0.0 & 20 & 0.0 & MVG & 0.22 & 0.17 & 0.41\\
LinReg5 & 0.0 & 5 & 0.01 & MVG & 0.23 & 0.18 & 0.44\\
LinReg6 & 0.0 & 5 & 0.01 & no noise & 0.29 & 0.29 & 0.29\\
LinReg22 & 0.01 & 5 & 0.01 & MVG & 0.30 & 0.28 & 0.43\\
LinReg4 & 0.0 & 5 & 0.1 & no noise & 0.30 & 0.30 & 0.30\\
LinReg7 & 0.1 & 5 & 0.0 & MVG & 0.34 & 0.30 & 0.41\\
LinReg19 & 0.01 & 5 & 0.1 & MVG & 0.35 & 0.31 & 0.45\\
LinReg29 & 0.0 & 20 & 0.01 & MVG & 0.36 & 0.30 & 0.48\\
LinReg13 & 0.1 & 5 & 0.01 & MVG & 0.39 & 0.35 & 0.53\\
LinReg31 & 0.1 & 20 & 0.0 & MVG & 0.41 & 0.38 & 0.45\\
LinReg27 & 0.0 & 20 & 0.1 & MVG & 0.41 & 0.31 & 0.51\\
LinReg10 & 0.1 & 5 & 0.1 & MVG & 0.41 & 0.34 & 0.51\\
LinReg8 & 0.1 & 5 & 0.0 & tracking noise & 0.42 & 0.39 & 0.46\\
LinReg37 & 0.1 & 20 & 0.01 & MVG & 0.42 & 0.42 & 0.49\\
Li

In [39]:
sort!(df, [:ks_best_replica])
print(df[!,["name", "tracking_noise",	"hist_len",	"lambda", "model_noise","summed_ks", "ks_best_replica", "ks_worst_replica"]])

48×8 DataFrame
 Row │ name      tracking_noise  hist_len  lambda   model_noise     summed_ks  ks_best_replica  ks_worst_replica 
     │ String    Float64         Int64     Float64  Symbol          Float64    Float64          Float64          
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ LinReg1             0.0          5     0.0   MVG              0.19812          0.141596          0.370516
   2 │ LinReg25            0.0         20     0.0   MVG              0.21629          0.169171          0.406965
   3 │ LinReg5             0.0          5     0.01  MVG              0.232814         0.175171          0.442689
   4 │ LinReg3             0.0          5     0.1   MVG              0.188315         0.215495          0.352016
   5 │ LinReg22            0.01         5     0.01  MVG              0.298098         0.278918          0.433639
   6 │ LinReg6             0.0          5     0.01  no_noise         0.286743 